In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
data=pd.read_excel('Bank_Personal_Loan_Modelling.xlsx',sheet_name=1)
data.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [3]:
data1=data.drop(['ID','ZIP Code'],axis=1)
data1.head()

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,25,1,49,4,1.6,1,0,0,1,0,0,0
1,45,19,34,3,1.5,1,0,0,1,0,0,0
2,39,15,11,1,1.0,1,0,0,0,0,0,0
3,35,9,100,1,2.7,2,0,0,0,0,0,0
4,35,8,45,4,1.0,2,0,0,0,0,0,1



**Check for null values**

In [4]:
data1.isna().sum()

Age                   0
Experience            0
Income                0
Family                0
CCAvg                 0
Education             0
Mortgage              0
Personal Loan         0
Securities Account    0
CD Account            0
Online                0
CreditCard            0
dtype: int64

**Splitting of dataset**



In [5]:
y=data1.iloc[:,7:8]
y.head()

,Personal Loan
0,0
1,0
2,0
3,0
4,0


In [6]:
x=data1.drop(['Personal Loan'],axis=1)
x.head()

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard
0,25,1,49,4,1.6,1,0,1,0,0,0
1,45,19,34,3,1.5,1,0,1,0,0,0
2,39,15,11,1,1.0,1,0,0,0,0,0
3,35,9,100,1,2.7,2,0,0,0,0,0
4,35,8,45,4,1.0,2,0,0,0,0,1


**Random forest algorithm and feature importance**

In [7]:
from sklearn.ensemble import RandomForestClassifier
rf_model=RandomForestClassifier(n_estimators=1000,max_features=2,oob_score=True)
rf_model.fit(x,y)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=2,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [8]:
print("OOB_accuracy is:",rf_model.oob_score_)

OOB_accuracy is: 0.9874


In [9]:

for x,imp in zip(x,rf_model.feature_importances_):
    print(x,imp);

Age 0.044054447354942246
Experience 0.04474197116361592
Income 0.3475448277902456
Family 0.10034155652937324
CCAvg 0.17977062828296883
Education 0.16150720067508725
Mortgage 0.04460791509152401
Securities Account 0.005384401042757949
CD Account 0.054471034289376524
Online 0.008055879932416762
CreditCard 0.009520137847691726


**Note**

Clearly Income,CCAvg,Education are most important features and decision tree will be created using these features only.

**Decision Tree**

In [10]:
from sklearn import tree
predictors=data1[['Income','CCAvg','Education']]
tree_model=tree.DecisionTreeClassifier(max_depth=8)
tree_model.fit(X=predictors,y=data1['Personal Loan'])

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=8, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')


**Creating a .dot file for the obtained decision tree**

In [12]:
with open('Dtree_loan.dot','w')as f:
    f=tree.export_graphviz(tree_model,feature_names=['Income','CCAvg','Education'],out_file=f);

**Leaf nodes inferences**

a) 3629 customers having income less than 106.5 dollars and CCavg less than 2.95 dollars did'nt get loan.

b) 314 customers having income greater than 116.5 dollars with education level of either Graduate or Advanced/Professional got the loan.

c) 3 customers having income greater than 106.5 dollars and less than 113.5 with education level of either Graduate or Advanced/Professional and CCavg less than 0.3 dollars didn't get the loan.

d) 22 customers having income greater than 109.5 dollars and less than 113.5 with education level of Undergraduate or and CCavg greater than 0.3 dollars and less than 2.95 dollars didn't get the loan.

e) 60 customers having income less than 81.5 dollars with CCavg greater than 3.55 dollars didn't get the loan.

f) 4 customers having income less than 82.5 dollars bnt greater than 81.5 dollars with CCavg greater than 3.75 dollars didn't get the loan.